# LangChain Expression Language (LCEL) with Amazon Nova

This notebook demonstrates chain composition patterns using LangChain Expression Language.

## Setup

In [ ]:
%env NOVA_API_KEY="YOUR-API-KEY"
%env NOVA_BASE_URL=https://api.nova.amazon.com/v1/

In [ ]:
from langchain_nova import ChatNova
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableParallel

# Initialize the model
llm = ChatNova(model="nova-pro-v1", temperature=0.7)

## 1. Simple Chain

The most basic LCEL pattern: `prompt | model | parser`

In [ ]:
# Create a simple chain
prompt = ChatPromptTemplate.from_template("Tell me a joke about {topic}")
chain = prompt | llm | StrOutputParser()

# Invoke it
result = chain.invoke({"topic": "programming"})
print(f"Result: {result}")

## 2. Sequential Chain

Chain multiple operations in sequence.

In [ ]:
# First step: translate
translate_prompt = ChatPromptTemplate.from_template(
    "Translate this to {language}: {text}"
)
translate_chain = translate_prompt | llm | StrOutputParser()

# Second step: summarize
summarize_prompt = ChatPromptTemplate.from_template(
    "Summarize in one sentence: {text}"
)
summarize_chain = summarize_prompt | llm | StrOutputParser()

# Test translation
translation = translate_chain.invoke({
    "text": "LangChain is great for building AI applications",
    "language": "Spanish"
})
print(f"Translation: {translation}\n")

# Then summarize the translation
summary = summarize_chain.invoke({"text": translation})
print(f"Summary: {summary}")

## 3. Parallel Execution

Run multiple chains in parallel using `RunnableParallel`.

In [ ]:
# Define two different chains
joke_chain = (
    ChatPromptTemplate.from_template("Tell a joke about {topic}")
    | llm
    | StrOutputParser()
)

poem_chain = (
    ChatPromptTemplate.from_template("Write a haiku about {topic}")
    | llm
    | StrOutputParser()
)

# Run them in parallel
parallel_chain = RunnableParallel(joke=joke_chain, poem=poem_chain)
results = parallel_chain.invoke({"topic": "clouds"})

print("Joke:")
print(results['joke'])
print("\nPoem:")
print(results['poem'])

## 4. Chain with Branching Logic

Use `RunnablePassthrough` to create more complex data flows.

In [ ]:
# Create a chain that analyzes and then elaborates
analyze_prompt = ChatPromptTemplate.from_template(
    "Analyze this text in 2-3 words: {text}"
)
elaborate_prompt = ChatPromptTemplate.from_template(
    "Original: {original}\nAnalysis: {analysis}\n\nElaborate on the analysis:"
)

# Chain that passes through original text while adding analysis
chain = (
    {"original": RunnablePassthrough(), "analysis": analyze_prompt | llm | StrOutputParser()}
    | elaborate_prompt
    | llm
    | StrOutputParser()
)

result = chain.invoke("The quick brown fox jumps over the lazy dog")
print(result)

## 5. Chain with Fallbacks

Add fallback behavior for robustness.

In [ ]:
# Primary model
primary = ChatNova(model="nova-pro-v1", temperature=0.7)

# Fallback model
fallback_model = ChatNova(model="nova-lite-v1", temperature=0.7)

# Create chain with fallback
prompt = ChatPromptTemplate.from_template("What is {thing}?")
chain_with_fallback = (
    prompt | primary | StrOutputParser()
).with_fallbacks([
    prompt | fallback_model | StrOutputParser()
])

result = chain_with_fallback.invoke({"thing": "LangChain"})
print(f"Result: {result}")

## 6. Streaming Chains

Stream output from chains token by token.

In [ ]:
prompt = ChatPromptTemplate.from_template(
    "Write a short story about {topic} in 3 sentences."
)
chain = prompt | llm | StrOutputParser()

print("Streaming output:")
for chunk in chain.stream({"topic": "a robot learning to paint"}):
    print(chunk, end="", flush=True)
print("\n")

## 7. Chain Composition with Data Transformation

Transform data between chain steps.

In [ ]:
# Function to transform data
def extract_keywords(text: str) -> dict:
    words = text.split()
    return {"keywords": ", ".join(words[:3])}

# Chain with transformation
keyword_prompt = ChatPromptTemplate.from_template(
    "Generate a title using these keywords: {keywords}"
)

chain = (
    extract_keywords
    | keyword_prompt
    | llm
    | StrOutputParser()
)

result = chain.invoke("artificial intelligence machine learning deep neural networks")
print(f"Generated title: {result}")

## Summary

**LCEL Chain Patterns:**

| Pattern | Syntax | Use Case |
|---------|--------|----------|
| Simple Chain | `prompt \| llm \| parser` | Basic transformations |
| Sequential | Multiple chains in order | Multi-step processing |
| Parallel | `RunnableParallel(...)` | Independent concurrent operations |
| Branching | `RunnablePassthrough()` | Complex data flows |
| Fallbacks | `.with_fallbacks([...])` | Error handling, redundancy |
| Streaming | `.stream(...)` | Real-time output |
| Transformation | Custom functions in chain | Data preprocessing |

**Key Advantages:**
- **Composable**: Build complex workflows from simple pieces
- **Readable**: Pipeline syntax makes logic clear
- **Flexible**: Easy to add/remove/modify steps
- **Async-ready**: All chains support async execution
- **Debuggable**: Each step can be tested independently